In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3"
os.environ["JAVA_HOME"] = "/usr/java/jdk-17.0.4/"
os.environ["SPARK_HOME"] = "/home/ec2-user/spark-3.3.0-bin-hadoop3"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.9.5-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

In [2]:
!pip3 install -q fsspec
!pip3 install -q s3fs
!pip3 install -q numpy
!pip3 install -q pandas

In [3]:
# SparkSession
from pyspark.sql import SparkSession

import numpy as np
import pandas as pd

In [4]:
spark = SparkSession.builder.master("local") \
                            .appName("Credit_Card_Applitcation_Approval") \
                            .config("spark.ui.port", "4050") \
                            .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/25 19:00:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df = pd.read_csv("s3a://decisiontreespark/fraud_detection_data.csv", header="infer")
data = spark.createDataFrame(df)
data.printSchema()

root
 |-- step: long (nullable = true)
 |-- type: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- nameOrig: string (nullable = true)
 |-- oldbalanceOrg: double (nullable = true)
 |-- newbalanceOrig: double (nullable = true)
 |-- nameDest: string (nullable = true)
 |-- oldbalanceDest: double (nullable = true)
 |-- newbalanceDest: double (nullable = true)
 |-- isFraud: long (nullable = true)
 |-- isFlaggedFraud: long (nullable = true)



In [6]:
data.show()

+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|    type|   amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|   1| PAYMENT|  9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|             0|
|   1| PAYMENT|  1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|             0|
|   1|TRANSFER|    181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|      1|             0|
|   1|CASH_OUT|    181.0| C840083671|        181.0|           0.0|  C38997010|       21182.0|           0.0|      1|             0|
|   1| PAYMENT| 11668.14|C2048537720|      41554.0|      29885.86|M123070170

In [7]:
# View columns.
data.columns

['step',
 'type',
 'amount',
 'nameOrig',
 'oldbalanceOrg',
 'newbalanceOrig',
 'nameDest',
 'oldbalanceDest',
 'newbalanceDest',
 'isFraud',
 'isFlaggedFraud']

In [8]:
# Select the required columns.
sel_cols=["step", "type", "amount", "oldbalanceOrg", "newbalanceOrig", 
          "oldbalanceDest", "newbalanceDest", "isFraud", "isFlaggedFraud"]

In [9]:
# Create a dataframe consisting of selected columns.
df = data.select(sel_cols)

In [10]:
df.show()

+----+--------+---------+-------------+--------------+--------------+--------------+-------+--------------+
|step|    type|   amount|oldbalanceOrg|newbalanceOrig|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+---------+-------------+--------------+--------------+--------------+-------+--------------+
|   1| PAYMENT|  9839.64|     170136.0|     160296.36|           0.0|           0.0|      0|             0|
|   1| PAYMENT|  1864.28|      21249.0|      19384.72|           0.0|           0.0|      0|             0|
|   1|TRANSFER|    181.0|        181.0|           0.0|           0.0|           0.0|      1|             0|
|   1|CASH_OUT|    181.0|        181.0|           0.0|       21182.0|           0.0|      1|             0|
|   1| PAYMENT| 11668.14|      41554.0|      29885.86|           0.0|           0.0|      0|             0|
|   1| PAYMENT|  7817.71|      53860.0|      46042.29|           0.0|           0.0|      0|             0|
|   1| PAYMENT|  7107.77|   

In [11]:
# View and group the data by “type” as follows:
df.groupby("type").count().show()

+--------+------+
|    type| count|
+--------+------+
|TRANSFER| 82424|
| CASH_IN|218673|
|CASH_OUT|362676|
| PAYMENT|329752|
|   DEBIT|  6474|
+--------+------+



In [12]:
# Explore the number of fraud cases.
df.groupby("isFraud").count().show()

+-------+------+
|isFraud| count|
+-------+------+
|      0|999464|
|      1|   535|
+-------+------+



In [13]:
# Import required libraries for string indexing.
from pyspark.ml.feature import StringIndexer

In [14]:
# Apply function for string indexer.
si = StringIndexer(inputCol="type", outputCol="type_index")

In [15]:
# Apply required transformation on dataframe.
df_indexed = si.fit(df).transform(df)

In [16]:
# View the new df.
df_indexed.show(5)

+----+--------+--------+-------------+--------------+--------------+--------------+-------+--------------+----------+
|step|    type|  amount|oldbalanceOrg|newbalanceOrig|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|type_index|
+----+--------+--------+-------------+--------------+--------------+--------------+-------+--------------+----------+
|   1| PAYMENT| 9839.64|     170136.0|     160296.36|           0.0|           0.0|      0|             0|       1.0|
|   1| PAYMENT| 1864.28|      21249.0|      19384.72|           0.0|           0.0|      0|             0|       1.0|
|   1|TRANSFER|   181.0|        181.0|           0.0|           0.0|           0.0|      1|             0|       3.0|
|   1|CASH_OUT|   181.0|        181.0|           0.0|       21182.0|           0.0|      1|             0|       0.0|
|   1| PAYMENT|11668.14|      41554.0|      29885.86|           0.0|           0.0|      0|             0|       1.0|
+----+--------+--------+-------------+--------------+---

In [17]:
# Import required libraries for one-hot encoder.
from pyspark.ml.feature import OneHotEncoder

In [18]:
# Apply function for one-hot encoder.
encoder = OneHotEncoder(inputCols=["type_index"], outputCols=["type_encoded"])

In [19]:
# Fit and transform your dataframe with the encoded data.
df_enc = encoder.fit(df_indexed)
df_encoded = df_enc.transform(df_indexed)

In [20]:
# View the new df.
df_encoded.show(3)

+----+--------+-------+-------------+--------------+--------------+--------------+-------+--------------+----------+-------------+
|step|    type| amount|oldbalanceOrg|newbalanceOrig|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|type_index| type_encoded|
+----+--------+-------+-------------+--------------+--------------+--------------+-------+--------------+----------+-------------+
|   1| PAYMENT|9839.64|     170136.0|     160296.36|           0.0|           0.0|      0|             0|       1.0|(4,[1],[1.0])|
|   1| PAYMENT|1864.28|      21249.0|      19384.72|           0.0|           0.0|      0|             0|       1.0|(4,[1],[1.0])|
|   1|TRANSFER|  181.0|        181.0|           0.0|           0.0|           0.0|      1|             0|       3.0|(4,[3],[1.0])|
+----+--------+-------+-------------+--------------+--------------+--------------+-------+--------------+----------+-------------+
only showing top 3 rows



In [21]:
# Import required libraries for vector assembler
from pyspark.ml.feature import VectorAssembler

In [22]:
# Select the columns required.
selected_cols=['step','amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest','type_encoded']

In [23]:
# Run the vector assembler function and save it into a new dataframe.
va = VectorAssembler(inputCols=selected_cols,outputCol="features")
df_vectors = va.transform(df_encoded)

In [24]:
df_vectors.show(5)

+----+--------+--------+-------------+--------------+--------------+--------------+-------+--------------+----------+-------------+--------------------+
|step|    type|  amount|oldbalanceOrg|newbalanceOrig|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|type_index| type_encoded|            features|
+----+--------+--------+-------------+--------------+--------------+--------------+-------+--------------+----------+-------------+--------------------+
|   1| PAYMENT| 9839.64|     170136.0|     160296.36|           0.0|           0.0|      0|             0|       1.0|(4,[1],[1.0])|(10,[0,1,2,3,7],[...|
|   1| PAYMENT| 1864.28|      21249.0|      19384.72|           0.0|           0.0|      0|             0|       1.0|(4,[1],[1.0])|(10,[0,1,2,3,7],[...|
|   1|TRANSFER|   181.0|        181.0|           0.0|           0.0|           0.0|      1|             0|       3.0|(4,[3],[1.0])|(10,[0,1,2,9],[1....|
|   1|CASH_OUT|   181.0|        181.0|           0.0|       21182.0|           0.0

In [25]:
# Divide the data into test and train data.
(trainingData, testData) = df_vectors.randomSplit([0.7,0.3])

In [26]:
# Run the decision tree classifier function.
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(featuresCol="features", labelCol="isFraud")

In [27]:
# Fit the training data.
model = dt.fit(trainingData)

In [28]:
# Transform the test data.
predictions = model.transform(testData)

In [29]:
# View the resultant dataframe to analyse better.
predictions.select("prediction", "isFraud", "features").show(5)

+----------+-------+--------------------+
|prediction|isFraud|            features|
+----------+-------+--------------------+
|       0.0|      0|[1.0,270.78,41849...|
|       0.0|      0|[1.0,783.31,81503...|
|       0.0|      0|[1.0,1076.27,3538...|
|       0.0|      0|[1.0,2673.64,7688...|
|       0.0|      0|[1.0,4865.48,7395...|
+----------+-------+--------------------+
only showing top 5 rows



In [30]:
# Load the required libraries.
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [31]:
# Run the evaluator function.
evaluator = MulticlassClassificationEvaluator(labelCol="isFraud",
                                              predictionCol="prediction",
                                              metricName="accuracy")

In [32]:
# View the accuracy.
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9996596289956052


In [33]:
# Load the required libraries.
from pyspark.sql.types import FloatType
from pyspark.mllib.evaluation import MulticlassMetrics
import pyspark.sql.functions as F

In [34]:
# Important: need to cast to float type, and order by prediction, else it won't work:

preds_and_labels = predictions.select(['prediction','isFraud']) \
                    .withColumn('label', F.col('isFraud').cast(FloatType())).orderBy('prediction')


In [35]:
# Select only prediction and label columns
preds_and_labels = preds_and_labels.select(['prediction','label'])

metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))

/home/ec2-user/spark-3.3.0-bin-hadoop3/python/lib/pyspark.zip/pyspark/sql/context.py:159: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.


In [36]:
np.set_printoptions(suppress=True)
print(metrics.confusionMatrix().toArray())

[[299484.     41.]
 [    61.     87.]]


In [37]:
# Precision for label 0
precision0 = metrics.precision(0)

# Recall for label 0
recall0 = metrics.recall(0)

# Precision for label 1
precision1 = metrics.precision(1)

# Recall for label 1
recall1 = metrics.recall(1)

# F1 score for label 0
f1Score0 = metrics.fMeasure(0.0, beta = 1.0)

# F1 score for label 1
f1Score1 = metrics.fMeasure(1.0, beta = 1.0)